In [3]:
import Pkg; Pkg.activate(".")

using ExcelReaders, DataFrames, Dates, Statistics, Unicode, StringEncodings, AnyAscii, CSV

  Activating environment at `E:\Documents\Dropbox\NYU\Proposals\Duration Mismatch\Project.toml`


# | Seperator

Firstly, save the files in UTF-8 in `Notepad`.

# Missing

`69485` is missing in 2010!

# Errors
* `63126` `1980 CSO ANB 4% NL 1994 CRF` has new business until 2007!

In [84]:
open("C24341_detail1.csv") do f
    g = open("C24341.csv", "w")

    s = readline(f)
    write(g, join(replace.(split(s, ","), "\"" => ""), "|") * "\n")

    while !eof(f)
        s = readline(f)
        sp1 = split(s, "\"")
        sp2 = split.(sp1[[1,3]], ",")
        str = vcat(sp2[1][1:2], sp1[2], sp2[2][2:4]..., join(sp2[2][5:end-1], ","), sp2[2][end])
        str = join(replace(str, "|" => " "), "|")
#         str = anyascii(str)

        if length(findall(isequal('|'), str)) == 7 && tryparse(Int64, sp2[2][end]) != nothing
            write(g, str * "\n")
        elseif length(findall(isequal('|'), str)) != 7
            @warn "$(length(findall(isequal('|'), str))): $str"
        end
    end

    close(g)

end

In [85]:
open("C24341_detail2.csv") do f
    g = open("C24341.csv", "a")

    s = readline(f)
#     write(g, join(replace.(split(s, ","), "\"" => ""), "|"))
    i = 1
    try

    while !eof(f)
        s = readline(f)
        i += 1
        sp1 = split(s, "\"")
        sp2 = split.(sp1[[1,3]], ",")
        str = vcat(sp2[1][1:2], sp1[2], sp2[2][2:4]..., join(sp2[2][5:end-1], ","), sp2[2][end])
        str = join(replace(str, "|" => " "), "|")
#         str = anyascii(str)

        if length(findall(isequal('|'), str)) == 7 && tryparse(Int64, sp2[2][end]) != nothing
            write(g, str * "\n")
        elseif length(findall(isequal('|'), str)) != 7
            @warn "$(length(findall(isequal('|'), str))): $str"
        end
    end
        catch err
        @warn string(err) * ": at $i"
    end
    close(g)

end

In [112]:
collect(eachmatch(r"\d{7}\.?(?!\d)",
    "1983 IAM at 6.00% Imm. & Def. 1999 20012002"))

1-element Vector{RegexMatch}:
 RegexMatch("0012002")

In [111]:
replace("WP TL ? 52 INTERCO DISA WITH 1980 CSO ANB 4.0% ? 4.5% NLP 2003 ? 2008", 
    r"((?<=\d%)|(?<=\d))\s?\?(?=\s?((\.|\d){2,4}|NB))" => " - ")

"WP TL ? 52 INTERCO DISA WITH 1980 CSO ANB 4.0% -  4.5% NLP 2003 -  2008"


# Cleaning Artifacts in Valuation Standard

## Errors
`0005,006253,"Presidential Life Insurance Company - USA",62200,2005,200009,1987       6.50%                                                                                 198,1602`

`1958 CSO 3.0$ NL`

`58 CSO CTF (^#)  3% LPM`

`CARVM 6-25% 87-91` and `CARVM 6-50% 89-89`

`83A 8.5% Deferred CARVM 84. 85`

`96 ANN 2000, 5 1/4% (2005-06. 2010)`

`0001,006341,"Equitable Financial Life Insurance Company",62944,2001,100047,58 CSO 3=%             (CRVM-CNF-ANB)ISL 1975-80,3697` and others from 62944 in 2001


## Things to clean up
`?` may be `...`, `-`, or ` `.

`0100007.`

`1983 GAM at 7 1/4% Imm. & Def. 8719901995` at 65099 2002.

`1963-1985................58 CET 2.5% NLP` at 58033 2002

`1994 GAR-6.25%  98-99 Immediate`

`1958 CSO 5.50%/20/3.50% CRVM ANB CRF 83` the first % is the correct rate

## How?

value.replace(/((?<=\d%)|(?<=\d))\s?\?(?=\s?((\.|\d){2,4}|NB))/, "-")

value.replace("?", "")

value.replace("", "")

value.replace(/^\d{7}(\s|\.)?/, "")

value.replace(/^\.(?=\d\d )/, "")

value.replace("`", "")

value.replace("'", "")

value.replace(/(?<=\d\d):(?=\d\d)/, "-")

value.replace(":", " ")

value.replace("@", " ")

value.replace("*", " ")

value.replace("_", " ")

value.replace(/\.{2,}/, " ")

value.replace(/(\.\s){2,}\.?/, " ")

Full dates: 

value.replace(/(0?[1-9]|1[012])([\/\-])(0?[1-9]|[12]\d|3[01])\2(?=\d{2,4})/, "")

Month dates with `/` and `yyyy`: 

value.replace(/(?<!\d\d)(?<!CLASS )(0?[1-9]|1[012])\/(?=\d{4})/, "")

Month dates with `/` and `yy`: 

value.replace(/(0?[1-9]|1[012])\/(?=\d\d-(0?[1-9]|1[012])\/\d\d)/, "")

value.replace(/(?<=\d\d-)(0?[1-9]|1[012])\/(?=\d\d)/, "")

Month dates with `-` and `yy`

value.replace(/(?<!\w)(?<!\d)(0?[1-9]|1[012])-(?=[3-9][0-9])/, "")

value.replace(/(?<=\d),(?=\d?\d%)/, ".")

value.replace("G.A. MORT.", "GAM")

value.replace("G.A.M.", "GAM")

value.replace("G.A. RES.", "GAR")

value.replace("I.A.M.", "IAM")

value.replace(/(?i)Immdediate/, "Immediate")

value.replace(/(?i)Immidiate/, "Immediate")

value.replace(/(?i)Immedicate/, "Immediate")

value.replace(/(?i)IMMEDATE/, "Immediate")
 
value.replace(/(?i)IMMEDIAATE/, "Immediate")

value.replace(/(?i)IMMEDDIATE/, "Immediate")

value.replace(/(?i)IMMEDAITE/, "Immediate")

value.replace(/(?i)Immediage/, "Immediate")

value.replace(/(?i)IMME?DI?(\;|\)|\(|\/|\d|,|\.|\s|$|&)/, "Immediate ")

value.replace(/(?i)IMM(\;|\)|\(|\/|\d|,|\.|\s|$|&)/, "Immediate ")

value.replace(/(?i)DEFANN/, "Deferred Annuity")

value.replace(/(?i)Deferreed/, "Deferred ")

value.replace(/(?i)DEFFERED/, "Deferred ")

value.replace(/(?i)DEFFERRED/, "Deferred ")

value.replace(/(?i)Deferrred/, "Deferred ")

value.replace(/(?i)DEFCARVM/, "Deferred CARVM")

value.replace(/(?i)DEFERED/, "Deferred ")

value.replace(/(?i)DEFImmediate/, "Deferred Immediate ")

value.replace(/(?i)DEFEFFERED/, "Deferred ")

value.replace(/(?i)DEFE?R?D?(\-|\;|\)|\(|\/|\d|,|\.|\s|$|&)/, "Deferred ")

value.replace("immediate", "Immediate")

value.replace("deferred", "Deferred")

value.replace(/^\d{1,3}\.\s/, "")

value.replace(/^\d{1,3}\s\./, "")

value.replace(/(?<=\d\.)\s(?=\d)/, "")

value.replace(/(?<=\d)\s(?=\.\d)/, "")

value.replace("S.A.", "SA")

value.replace("C.S.O.", "CSO")

value.replace("G.A.R.", "GAR")

value.replace("N.B.", "NB")

value.replace("A.E", "AmExp")

value.replace(/A\.\s?E\./, "AmExp")

value.replace(/N\.\s?L\./, "NL")

value.replace(/(?i)STD\.\s?IND\./, "StdInd")

value.replace(/(?i)Am\.?\s?Exp\.?/, "AmExp")

value.replace(/(?i)Amer\.?\s?Exp\.?/, "AmExp")

value.replace("Cont.", "Cont")

value.replace("Curt.", "Curt")

value.replace("InterCo.", "InterCo")

value.replace(/(?i)CERT\. ONLY/, "CertOnly")

value.replace(/(?i)InterCo\./, "InterCo")

value.replace(/(?i)C\.\s?E\.\s?T\./, "CET")

value.replace(/N\.\s?Y\./, "NY")

value.replace(/A\.\s?L\.\s?B\./, "ALB")

value.replace(/C\.\s?R\.\s?V\.\s?M\./, "CRVM")

value.replace(/(?i)ILL\./, "Ill")

value.replace(/(?i)I\.\s?S\./, "IS")

value.replace(/(?i)S\.\s?P\./, "SP")

value.replace(/(?i)AMERICAN EXPER\./, "AmExp")

value.replace(/(?i)AMERICAN EXP\./, "AmExp")

value.replace(/[ï¿½]{2,}/, "")

value.replace("½", " 1/2")

value.replace(/[^\x00-\x7F]/, " ")

value.replace("(", "")

value.replace(")", "")

value.replace(/(?i)\s?\-\-?\s?(?=[a-mo-z])/, " ")

value.replace(/(?i)(?<=[a-z])\s?\-\/?\s?/, "")

value.replace("- NLP", " NLP")

value.replace(/(?<=\d)\s1\/2\s?%?/, ".5%")

value.replace(/(?<=\d)\s1\/4\s?%?/, ".25%")

value.replace(/(?<=\d)\s3\/4\s?%?/, ".75%")

value.replace(/(?<=\d)\s1\/5\s?%?/, ".2%")

value.replace(/(?<=\d)\s3\/8\s?%?/, ".375%")

value.replace(/(?<=\d)\s1\/10\s?%?/, ".1%")

value.replace(/(?<=\d)\s3\/10\s?%?/, ".3%")

value.replace(/(?<=\d)\s6\/10\s?%?/, ".6%")

value.replace(/(?<=\d)\s9\/10\s?%?/, ".9%")

value.replace(/(o=){2,}o?/, " ")

value.replace(/\s{2,}/, " ")

value.replace(/(?<=\d)\s\-\s(?=\d)/, "-")

# Glossary

* Valuation Methods: `CRVM`, `NL`, `Mod`
* `NLP` is net level premium
* `SP` is single premium
* `ANB` and `ALB` are age nearest birthdaty and last
* `ISL` is interest sensitive life contracts
* `SS` structured settlement, `FI` fixed income, `NFI` annuity income not yet determined

* Life Insurance only: `CNF`, `IPC`, and `CRF` is continuous functions, immediate payment of claims, and curatate functions.

* `FPDA` flexible premium deferred annuity
* `3.5%/10/2.5%` means 3.5% for 10 years then 2.5%
* `CIF` is Collective Investment Fund
* `MDBG` minimum death benefit guarantee
* `FPT` is full preliminary term reserve valuation
* `Ill` is Illinois
* `ET` is experience or extended table
* `NPV` is net present value method

## Valuation Laws

* `AG33` is about non-elective non-mortality benefits, like disability and long-term care
* `AG34` is about variable annuities and minimum guaranteed death benefits
* `AG43` is about variable annuities
* `VM21` is about variable annuities
* `VM22` is the post-2017 reserve rate calculation for annuities

## Law

`(?i)((19)?41\s?CSO(?!\s?(19)?41)|(?<!41\s)CSO\s?(19)?41)` 
`1941CSO`
`value.replace(/(?i)((19)?41\s?CSO(?!\s?(19)?41)|(?<!41\s)CSO\s?(19)?41)/, "")`

`(?i)((19)?58\s?CSO(?!\s?(19)?58)|(?<!58\s)CSO\s?(19)?58)` 
`1958CSO`
`value.replace(/(?i)((19)?58\s?CSO(?!\s?(19)?58)|(?<!58\s)CSO\s?(19)?58)/, "")`

`(?i)((19)?80\s?CSO(?!\s?(19)?80)|(?<!80\s)CSO\s?(19)?80)` 
`1980CSO`
`value.replace(/(?i)((19)?80\s?CSO(?!\s?(19)?80)|(?<!80\s)CSO\s?(19)?80)/, "")`

`(?i)((20)?01\s?CSO(?!\s?(20)?01)|(?<!01\s)CSO\s?(20)?01)` 
`2001CSO`
`value.replace(/(?i)((20)?01\s?CSO(?!\s?(20)?01)|(?<!01\s)CSO\s?(20)?01)/, "")`

`(?i)((20)?17\s?CSO(?!\s?(20)?17)|(?<!17\s)CSO\s?(20)?17)` 
`2017CSO`
`value.replace(/(?i)((20)?17\s?CSO(?!\s?(20)?17)|(?<!17\s)CSO\s?(20)?17)/, "")`

`(?i)((19)?41\s?CET(?!\s?(19)?41)|(?<!41\s)CET\s?(19)?41)` 
`1941CET`
`value.replace(/(?i)((19)?41\s?CET(?!\s?(19)?41)|(?<!41\s)CET\s?(19)?41)/, "")`

`(?i)((19)?58\s?CET(?!\s?(19)?58)|(?<!58\s)CET\s?(19)?58)` 
`1958CET`
`value.replace(/(?i)((19)?58\s?CET(?!\s?(19)?58)|(?<!58\s)CET\s?(19)?58)/, "")`

`(?i)((19)?61\s?CET(?!\s?(19)?61)|(?<!61\s)CET\s?(19)?61)` 
`1961CET`
`value.replace(/(?i)((19)?61\s?CET(?!\s?(19)?61)|(?<!61\s)CET\s?(19)?61)/, "")`

`(?i)((19)?80\s?CET(?!\s?(19)?80)|(?<!80\s)CET\s?(19)?80)` 
`1980CET`
`value.replace(/(?i)((19)?80\s?CET(?!\s?(19)?80)|(?<!80\s)CET\s?(19)?80)/, "")`

`(?i)((20)?01\s?CET(?!\s?(20)?01)|(?<!01\s)CET\s?(20)?01)` 
`2001CET`
`value.replace(/(?i)((20)?01\s?CET(?!\s?(20)?01)|(?<!01\s)CET\s?(20)?01)/, "")`

`(?i)((19)?61\s?CSI(?!\s?(19)?61)|(?<!61\s)CSI\s?(19)?61)` 
`1961CSI`
`value.replace(/(?i)((19)?61\s?CSI(?!\s?(19)?61)|(?<!61\s)CSI\s?(19)?61)/, "")`

65056 has errors:

`value.replace(/^\s*1958/, "")`
`1958CSO`

`value.replace(/^\s*1980/, "")`
`1980CSO`

`(?i)((19)?41\s?SSI(?!\s?(19)?41)|(?<!41\s)SSI\s?(19)?41)` 
`1941SSI`
`value.replace(/(?i)((19)?41\s?SSI(?!\s?(19)?41)|(?<!41\s)SSI\s?(19)?41)/, "")`

`(?i)((19)?41\s?SI(?!\s?(19)?41)|(?<!41\s)SI\s?(19)?41)` 
`1941SI`
`value.replace(/(?i)((19)?41\s?SI(?!\s?(19)?41)|(?<!41\s)SI\s?(19)?41)/, "")`

`(?i)((19)?61\s?CIET(?!\s?(19)?61)|(?<!61\s)CIET\s?(19)?61)` 
`1961CIET`
`value.replace(/(?i)((19)?61\s?CIET(?!\s?(19)?61)|(?<!61\s)CIET\s?(19)?61)/, "")`

`(?i)((19)?55\s?Am.*ann(uity)?\.?(?!\s?(19)?55)|(?<!55\s)Am.*ann(uity)?\.?\s?(19)?55)` 
`1955AmAnn`
`value.replace(/(?i)((19)?55\s?Am.*ann(uity)?\.?(?!\s?(19)?55)|(?<!55\s)Am.*ann(uity)?\.?\s?(19)?55)/, "")`

`(?i)((19)?41\s?Std.*ind\.?(?!\s?(19)?41)|(?<!41\s)Std.*ind\.?\s?(19)?41)` 
`1941StdInd`
`value.replace(/(?i)((19)?41\s?Std.*ind\.?(?!\s?(19)?41)|(?<!41\s)Std.*ind\.?\s?(19)?41)/, "")`

`(?i)((19)?85\s?ja?pa?n(?!\s?(19)?85)|(?<!85\s)ja?pa?n\s?(19)?85)` 
`1985Japan`
`value.replace(/(?i)((19)?85\s?ja?pa?n(?!\s?(19)?85)|(?<!85\s)ja?pa?n\s?(19)?85)/, "")`

`(?i)((19)?96\s?ja?pa?n(?!\s?(19)?96)|(?<!96\s)ja?pa?n\s?(19)?96)` 
`1996Japan`
`value.replace(/(?i)((19)?96\s?ja?pa?n(?!\s?(19)?96)|(?<!96\s)ja?pa?n\s?(19)?96)/, "")`

`(?i)((20)?07\s?ja?pa?n(?!\s?(20)?07)|(?<!07\s)ja?pa?n\s?(20)?07)` 
`2007Japan`
`value.replace(/(?i)((20)?07\s?ja?pa?n(?!\s?(20)?07)|(?<!07\s)ja?pa?n\s?(20)?07)/, "")`

`(?i)((19)?41\s?(ny\s)?st(andar)?d\.?\s?ind(ustrial)?\.?(?!\s?(19)?41)|(?<!41\s)(ny\s)?st(andar)?d\.?\s?ind(ustrial)?\.?\s?(19)?41)` 
`1941StdInd`
`value.replace(/(?i)((19)?41\s?(ny\s)?st(andar)?d\.?\s?ind(ustrial)?\.?(?!\s?(19)?41)|(?<!41\s)(ny\s)?st(andar)?d\.?\s?ind(ustrial)?\.?\s?(19)?41)/, "")`

Japan 



`(?i)((19)?51\s?GAM(?!\s?(19)?51)|(?<!51\s)GAM\s?(19)?51)` 
`1951GAM`
`value.replace(/(?i)((19)?51\s?GAM(?!\s?(19)?51)|(?<!51\s)GAM\s?(19)?71)/, "")`

`(?i)((19)?71\s?IAM(?!\s?(19)?71)|(?<!71\s)IAM\s?(19)?71)` 
`1971IAM`
`value.replace(/(?i)((19)?71\s?IAM(?!\s?(19)?71)|(?<!71\s)IAM\s?(19)?71)/, "")`

`(?i)((19)?71\s?GAM(?!\s?(19)?71)|(?<!71\s)GAM\s?(19)?71)` 
`1971GAM`
`value.replace(/(?i)((19)?71\s?GAM(?!\s?(19)?71)|(?<!71\s)GAM\s?(19)?71)/, "")`

`(?i)((19)?83\s?IAM(?!\s?(19)?83)|(?<!83\s)IAM\s?(19)?83)` 
`1983IAM`
`value.replace(/(?i)((19)?83\s?IAM(?!\s?(19)?83)|(?<!83\s)IAM\s?(19)?83)/, "")`

`(?i)((19)?83\s?GAM(?!\s?(19)?83)|(?<!83\s)GAM\s?(19)?83)` 
`1983GAM`
`value.replace(/(?i)((19)?83\s?GAM(?!\s?(19)?83)|(?<!83\s)GAM\s?(19)?83)/, "")`

`(?i)(19)?83\s?A`
`1983a`
`value.replace(/(?i)(19)?83\s?A/, "")`

`(?i)((19)?94\s?GAR(?!\s?(19)?94)|(?<!94\s)GAR\s?(19)?94)` 
`1994GAR`
`value.replace(/(?i)((19)?94\s?GAR(?!\s?(19)?94)|(?<!94\s)GAR\s?(19)?94)/, "")`

`(?i)((19)?94\s?GAM(?!\s?(19)?94)|(?<!94\s)GAM\s?(19)?94)` 
`1994GAM`
`value.replace(/(?i)((19)?94\s?GAM(?!\s?(19)?94)|(?<!94\s)GAM\s?(19)?94)/, "")`

`(?i)((20)?00\s?\-?A(nn)?(uity)?(?!\s?(20)?00)|(?<!00\s)A\-?(nn)?(uity)?\s?(20)?00)` 
`2000Ann`
`value.replace(/(?i)((20)?00\s?\-?A(nn)?(uity)?(?!\s?(20)?00)|(?<!00\s)A\-?(nn)?(uity)?\s?(20)?00)/, "")`

`(?i)((20)?00\s?IAM(?!\s?(20)?00)|(?<!00\s)IAM\s?(20)?00)` 
`2000IAM`
`value.replace(/(?i)((20)?00\s?IAM(?!\s?(20)?00)|(?<!00\s)IAM\s?(20)?00)/, "")`

`(?i)((20)?12\s?IAM(?!\s?(20)?12)|(?<!12\s)IAM\s?(20)?12)` 
`2012IAM`
`value.replace(/(?i)((20)?12\s?IAM(?!\s?(20)?12)|(?<!12\s)IAM\s?(20)?12)/, "")`

`(?i)((20)?12\s?IAR(?!\s?(20)?12)|(?<!12\s)IAR\s?(20)?12)` 
`2012IAR`
`value.replace(/(?i)((20)?12\s?IAR(?!\s?(20)?12)|(?<!12\s)IAR\s?(20)?12)/, "")`

`a-(19)?49`
`1949a`
`value.replace(/a-(19)?49/, "")`

`(?i)((20)?12\s?\-?A(nn)?(uity)?(?!\s?(20)?12)|(?<!12\s)A\-?(nn)?(uity)?\s?(20)?12)`
`2012IAM`
`value.replace(/(?i)((20)?12\s?\-?A(nn)?(uity)?(?!\s?(20)?12)|(?<!12\s)A\-?(nn)?(uity)?\s?(20)?12)/, "")`

`(19)?83 TA?BLE? A`
`1983a`
`value.replace(/(?i)(19)?83 TA?BLE? A/, "")`

`value.replace(/(?i)ACCOUNT VALUE/, "")`
`value.replace(/(?i)FUND ACCUMULATIONS/, "")`

`(?i)((19)?66\s?GAM(?!\s?(19)?66)|(?<!66\s)GAM\s?(19)?66)` 
`1966GAM`
`value.replace(/(?i)((19)?66\s?GAM(?!\s?(19)?66)|(?<!66\s)GAM\s?(19)?66)/, "")`

`value.replace(/(?i)(19)?71 Individual Annuity/, "")`

`value.replace(/(?i)(19)?49 Individual Annuity/, "")`

`value.replace(/(?i)(19)?83 Individual Annuity/, "")`

`value.replace(/(?i)a200\s?(INDIVIDUAL ANNUITY)?/, "")`

`value.replace(/(?i)(19)?71 Individual annuities/, "")`

`value.replace(/(?i)(19)?83 Individual annuities/, "")`

`value.replace(/(?i)(20)?12 Individual annuities/, "")`

`value.replace(/(?i)(19)?71 Ind(ividual)?\.? ann(uit)?(y|ies)?/, "")`

`value.replace(/(?i)(19)?83 Ind(ividual)?\.? ann(uit)?(y|ies)?/, "")`

`value.replace(/(?i)(20)?00 Ind(ividual)?\.? ann(uit)?(y|ies)?/, "")`

`value.replace(/(?i)(20)?12 Ind(ividual)?\.? ann(uit)?(y|ies)?/, "")`

`(?i)((19)?51\s?Gr(ou)?p\.? ann(uit)?(y|ies)?(?!\s?(19)?51)|(?<!51\s)Gr(ou)?p\.? ann(uit)?(y|ies)?\s?(19)?51)` 
`1951GAM`
`value.replace(/(?i)((19)?51\s?Gr(ou)?p\.? ann(uit)?(y|ies)?(?!\s?(19)?51)|(?<!51\s)Gr(ou)?p\.? ann(uit)?(y|ies)?\s?(19)?51)/, "")`

`(?i)((19)?37\s?St?(andar)?d?\.?\s?A(nn)?(uity)?\.?(?!\s?(19)?51)|(?<!37\s)St?(andar)?d?\.?\s?A(nn)?(uity)?\.?\s?(19)?51)` 
`1937SA`
`value.replace(/(?i)((19)?37\s?St?(andar)?d?\.?\s?A(nn)?(uity)?\.?(?!\s?(19)?51)|(?<!37\s)St?(andar)?d?\.?\s?A(nn)?(uity)?\.?\s?(19)?51)/, "")`

`value.replace(/(?i)AM 2000/, "")`

`value.replace(/(?i)a1983/, "")`

`value.replace(/(?i)A1949/, "")`

`value.replace(/(?i)(19)?94 V(ariable)?\s?\.?A(nn)?(uit)?(y|ies)?\.?/, "")`

`value.replace(/(?i)G\. A\. 51/, "")`

`value.replace(/(?i)1983 IAV/, "")`

`value.replace(/(?i)INT(erest)?\.? ONLY/, "")`

`value.replace(/(?i)(19)?83 IAV/, "")`

`value.replace(/(?i)(19)?83 IA/, "")`

`value.replace(/(?i)(19)?83 IA/, "")`

`value.replace(/(?i)(19)?84 TA?BLE? A/, "")`

`value.replace(/(?i)(19)?83 GR?P\.? ANN/, "")`

`value.replace(/(?i)^2000/, "")` for 66869

`value.replace(/(?i)2000 (IA|TA?BLE?)/, "")`

`value.replace(/(?i)^(20)?00/, "")` for some, check!

`value.replace(/(?i)(19)?94 Gro?u?p\.? Ann(uity)?\.? Res(erv(ing|e))\.?/, "")`

`value.replace(/(?i)^A 1949/, "")`

`value.replace(/(?i)(19)?49 ANN(uity)?/, "")`

`value.replace(/(?i)ANNUITY 2001/, "")`

`value.replace(/(?i)AG\s?43(\svar(iable)?\.?\sann(uit(y|ies)))?/, "")`

`value.replace(/(?i)VM\s?21/, "")`

`value.replace(/(?i)VM\s?22/, "")`

`value.replace(/(?i)VM\s?20/, "")`

`value.replace(/(?i)(19)?83 Gr?p?\.? A(nn)?\.? MORT(ality)?\.?/, "")`

`value.replace(/(?i)(19)?60\s?CSG/, "")`

`value.replace(/(?i)GRP ANN TABLE FOR 1951/, "")`

`value.replace(/(?i)(19)?94 MGDB/, "MGDB")`

`value.replace(/(?i)(19)?94VAM/, "")`

`value.replace(/(?i)(19)?70|s?INTERCO/, "")`

`(?i)((19)?52\s?INTERCO(?!\s?(19)?52)|(?<!52\s)INTERCO\s?(19)?52)` 
`1952Interco`
`value.replace(/(?i)((19)?52\s?INTERCO(?!\s?(19)?52)|(?<!52\s)INTERCO\s?(19)?52)/, "")`

`value.replace(/(?i)AM EXP(erience)?/, "")`

`value.replace(/(?i)AM MEN/, "")`

`value.replace(/(?i)(19)?59 ADB/, "")`

`value.replace(/(?i)AM5/, "")`

`value.replace(/(19)?80 CS0/, "")`

`value.replace(/(?i)A49/, "")`

`value.replace(/(?i)(19)?49\s?A/, "")`

`value.replace(/(?i)A\s?(19)?49/, "")`

`value.replace(/(?i)SA\s?(19)?37/, "")`

`value.replace(/(?i)(19)?37\s?An?(nuity)?a?/, "")`

`value.replace(/(?i)(19)?70(\smodified\s)?interco/, "")`

`value.replace(/(?i)(19)?51 GA/, "")`

`value.replace(/(?i)(19)?06 S(td\.)?\s?I(nd)?\.?/, "")`

`value.replace(/(?i)(19)?83 INDIV ANNUITY/, "")`

`value.replace(/(?i)(19)?83 GROUP ANNUITY/, "")`

`value.replace(/(?i)(19)?71 GROUP ANNUITY/, "")`

`value.replace(/(?i)(19)?31 GAM/, "")`

`value.replace(/(?i)^AM\s5?/, "")`

`value.replace(/(?i)^(19)?83\s(table|ga)\s/, "")`

`value.replace(/(?i)(19)?83 DEFERRED I\.A\.M/, "")`

`value.replace("1983 DEFERRED G.A.M", "Deferred")`

`value.replace("1971 DEFERRED I.A.M", "Deferred")`

`value.replace(/(19)?71 I\.A\.M/, "")`

`value.replace(/(19)?71 I AM/, "")`

`value.replace(/(19)?41 CSI/, "")`

`value.replace(/(19)?41 STD\.?\sINT\.?/, "")`

`value.replace(/^(19)?83 I\. A\. MORT\.?/, "")`

`value.replace(/83G/, "")`

`value.replace(/1983 INDIV/, "")`

`value.replace(/50\/50 83/, "")`

`value.replace("1983 I.A. MORT.", "")`

`value.replace(/(19)?83 I\.\s?A\.\s?MORT/, "")`

`value.replace(/(19)?71\s?I\.?\s?A\.?\s?MORT\.?/, "")`

`value.replace(/(19)?71\s?G\.?\s?A\.?\s?MORT\.?/, "")`

`value.replace(/(19)?71\s?G\.?\s?A\.?\s?M\.?/, "")`

`value.replace(/(19)?83\s?G\.?\s?A\.?\s?MORT\.?/, "")`

`value.replace(/(19)83 G\.?\s?A\.?\s?M\.?/, "")`

`value.replace(/(19)?71 G\.?\s?A\.?/, "")`

`value.replace(/G\.?\s?A\.?\s?(19)?71/, "")`

`value.replace(/(?i)(19)?71 Ind(iv)?\s?\.?(ann(uity)?)?/, "")`

`value.replace(/(?i)(19)?71 IA(?![a-z])/, "")`

`value.replace(/(?i)(19)?71 GR?P ANN/, "")`

`value.replace(/(?i)EQ 1971 Life Inc\./, "")`

`value.replace(/(?i)EQ 1963 life inc\./, "")`

`value.replace(/(?i)EQ 1954 life inc\./, "")`

`value.replace(/(?i)(19)?71\sA\s/, "")`

`value.replace(/(?i)(19)?71A\s/, "")`

`value.replace(/(?i)(19)?71\s?Annuity\stable/, "")`

`value.replace(/(?i)(19)?71 MOD IAM/, "Mod")`

`value.replace("1971 DEFERRED G.A.M", "Deferred")`

`value.replace(/(?i)^(20)?12 (table|ia)/, "")`

`value.replace(/^(?i)(19)?37 (IAM|std)/, "")`

`value.replace(/^(?i)(19)?94 G\.?\s?A\.?\s?(RESV)?\.?/, "")`

`value.replace(/(?i)^(19)?94 MDGB /, "")`

`value.replace(/(?i)^(19)?94 GMDB /, "")`

`value.replace(/(?i)^(19)?94 Minimum Guarantee Death Benefit Table/, "MGDB")`

`value.replace(/(?i)^1983\sG/, "")`






SPI.A. - 71 IA83 - 11.25%


`if(cells['Valuation Standard'].value.contains(/(?i)(19)?41 CSO/), "1941 CSO", "")` 
`value.replace(/(?i)(19)?41 CSO/, "")`

## Rate

`cells['Valuation Standard'].value.find(/((\d?\d\.\d\d?)|((?<!\d)\d?\d))\s?%/)[0]`

`value.replace(/((\d?\d\.\d\d?)|((?<!\d)\d?\d))\s?%/, "")`

filter blank

`cells['Valuation Standard'].value.find(/\d?\d\.\d\d?/)[0]`

`cells['Valuation Standard'].value.find(/\d?\d\.\d\d?/)[1]`

`value.replace(/\d?\d\.\d\d?/, "")`

filter blank

`cells['Valuation Standard'].value.find(/(?<=\s)\d(?=\s)/)[0]`

`value.replace(/(?<=\s)\d(?=\s)/, "")`

`cells['Valuation Standard'].value.find(/\d\.%/)[0]`

`value.replace(/\d\.%/, "")`

# Def/Ann

`cells['Valuation Standard'].value.find(/(?i)Deferred/)[0]`

`value.replace(/(?i)Deferred/, "")`

`cells['Valuation Standard'].value.find(/(?i)immediate/)[0]`

`value.replace(/(?i)immediate/, "Both")`

`value.replace(/(?i)immediate/, "")`



# Valuation Method

`cells['Valuation Standard'].value.find(/(?i)CRVM/)[0]`

`value.replace(/(?i)CRVM/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)net level($|\s|P|\spremium)/))==1, if(length(cells['Method'].value) > 0, "various", "NLP"), cells['Method'].value)`

`value.replace(/(?i)net level($|\s|P|\spremium)/, "")`

# Bday

`if(length(cells['Valuation Standard'].value.find(/(?i)ALB/))==1, if(length(cells['Bday'].value) > 0, "various", "ALB"), cells['Bday'].value)`

`value.replace(/(?i)ALB/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)ANB/))==1, if(length(cells['Bday'].value) > 0, "various", "ANB"), cells['Bday'].value)`

`value.replace(/(?i)ANB/, "")`

# AddLife

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))CNF(?=\s|$)/))==1, if(length(cells['AddLife'].value) > 0, "various", "CNF"), cells['AddLife'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))CNF(?=\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))CRF(?=\s|$)/))==1, if(length(cells['AddLife'].value) > 0, "various", "CRF"), cells['AddLife'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))CRF(?=\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))IPC(?=\s|$)/))==1, if(length(cells['AddLife'].value) > 0, "various", "IPC"), cells['AddLife'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))IPC(?=\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))CURT?(ate)?(?=\s|$)/))==1, if(length(cells['AddLife'].value) > 0, "various", "CRF"), cells['AddLife'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))CURT?(ate)?(?=\s|$)/, "")`

# Sex

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))M(\s|$|ale)/))==1, if(length(cells['Sex'].value) > 0, "both", "male"), cells['Sex'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))M(\s|$|ale)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))F(\s|$|emale)/))==1, if(length(cells['Sex'].value) > 0, "both", "female"), cells['Sex'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))F(\s|$|emale)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))UNISEX/))==1, if(length(cells['Sex'].value) > 0, "both", "unisex"), cells['Sex'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))UNISEX/, "")`

# OtherMod

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))NJ(\s|$)/))==1, cells['OtherMod'].value + "+NJ", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))NJ(\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))IS(\s|$)/))==1, cells['OtherMod'].value + "+IS", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))IS(\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))Mod(ified)?(\s|$)/))==1, cells['OtherMod'].value + "+Mod", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))Mod(ified)?(\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))Std(\s|$)/))==1, cells['OtherMod'].value + "+Std", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))Std(\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))Ill(\s|$)/))==1, cells['OtherMod'].value + "+Ill", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))Ill(\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))IDB(\s|$)/))==1, cells['OtherMod'].value + "+IDB", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))IDB(\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))FPT(\s|$)/))==1, cells['OtherMod'].value + "+FPT", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))FPT(\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))NJS(\s|$)/))==1, cells['OtherMod'].value + "+NJS", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))NJS(\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))NSP(\s|$)/))==1, cells['OtherMod'].value + "+NSP", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))NSP(\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))ETI(\s|$)/))==1, cells['OtherMod'].value + "+ETI", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))ETI(\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))NSP(\s|$)/))==1, cells['OtherMod'].value + "+NSP", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))NSP(\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))(PREMIUM )?P(AI)?D\s?UP(\s|$)/))==1, cells['OtherMod'].value + "+PaidUp", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))(PREMIUM )?P(AI)?D\s?UP(\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))(PREMIUM )?PAYING(\s|$)/))==1, cells['OtherMod'].value + "+Paying", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))(PREMIUM )?PAYING(\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))(SEMI\s?)?CONT(inuous)?(\s|$)/))==1, cells['OtherMod'].value + "+Cont", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))(SEMI\s?)?CONT(inuous)?(\s|$)/, "")`

`cells['OtherMod'].value + "+" + cells['Valuation Standard'].value.partition(/(?i)((?<=\s)|(?<=^))\d\d\d%(\s|$)/)[1]`

`value.replace(/(?i)((?<=\s)|(?<=^))\d\d\d%(\s|$)/, "")`

((?<=\s)|(?<=^))\/\d\d/(?=\s)

`value.replace(/(?i)((?<=\s)|(?<=^))\/\d\d\/(?=\s)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))GC(\s|$)/))==1, cells['OtherMod'].value + "+GC", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))GC(\s|$)/, "")`

### StepRate `\d?\d\.?\d?\d?%?\/\d\d\/\d?\d\.?\d?\d?%`

`value.replace(/(?i)((?<=\s)|(?<=^))Blended(?=\s)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))S(EL)?(ect)?\.?\s?(\/|\&|and)\s?U(LT)?\.?(IMATE)?(\s|$)/))==1, cells['OtherMod'].value + "+Sel&Ult", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))S(EL)?(ect)?\.?\s?(\/|\&|and)\s?U(LT)?\.?(IMATE)?(\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))S(EL)?(ect)?\.?(\s|$)/))==1, cells['OtherMod'].value + "+Sel", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))S(EL)?(ect)?\.?(\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))U(LT)?\.?(IMATE)?(\s|$)/))==1, cells['OtherMod'].value + "+Ult", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))U(LT)?\.?(IMATE)?(\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))SM?(\s|$)/))==1, cells['OtherMod'].value + "+Smoker", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))SM?(\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))NS?M?(\s|$)/))==1, cells['OtherMod'].value + "+NonSmoker", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))NS?M?(\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))Non\-?smoker\s?\/\s?Smoker/))==1, cells['OtherMod'].value + "+Unismoke", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))Non\-?smoker\s?\/\s?Smoker/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))Unismoker?/))==1, cells['OtherMod'].value + "+Unismoke", cells['OtherMod'].value)`

`value.replace(/(?i)Unismoker?/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))S(MK)?\s?\/\s?NSM?/))==1, cells['OtherMod'].value + "+Unismoke", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))S(MK)?\s?\/\s?NSM?/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))NJS(\s|$)/))==1, cells['OtherMod'].value + "+NJS", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))NJS(\s|$)/, "")`

value.replace(/(?i)NL /, "")
value + "+Mod"

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))MPT(\s|$)/))==1, cells['OtherMod'].value + "+MPT", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))MPT(\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))ISL(\s|$)/))==1, cells['OtherMod'].value + "+ISL", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))ISL(\s|$)/, "")`

`value.replace(/(?i)S\s?\&\s?U/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))ON(\s|$)/))==1, cells['OtherMod'].value + "+ON", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))ON(\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))IL(\s|$)/))==1, cells['OtherMod'].value + "+Ill", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))IL(\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((\d\d)|\s|\,){1,}/))==1, cells['OtherMod'].value + "+Ill", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))IL(\s|$)/, "")`

`value.replace(/(?i)((?<=\s)|(?<=^))UL(\s|$)/, "")`

`value.replace(/(?i)((?<=\s)|(?<=^))VUL(\s|$)/, "")`

`value + "+VUL"`

`value.replace(/(?i)U\/L/, "")`

`value.replace(/(?i)((?<=\s)|(?<=^))standard(\s|$)/, "")`

`value.replace(/(?i)((?<=\s)|(?<=^))NLVM(\s|$)/, "")`

`value.replace(/(?i)((?<=\s)|(?<=^))Non(\s|\-)Smoker(\s|$)/, "")`

`value + "+VUL"`

`value.replace(/(?i)((?<=\s)|(?<=^))N\/S(\s|$)/, "")`

`value.replace(/(?i)((?<=\s)|(?<=^))F\s?\/\s?NS(\s|$)/, "")`

130%
1958 Commissioners Standard Ordinary
10 /

30 / 

`value.replace(/(?i)(?<=\d)\s?\-\s?(?=\d|$)/, "-")`

# Years

`cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))(((\d\d)?\d\d)|\-|\,|\s|\&|later|than|before|since|prior|after|earlier|to|and)*((\d\d)|NB)(((\d\d)?\d\d)|\-|\,|\s|NB|\&|later|before|since|prior|after|earlier|to|and|date|present)*(\s|$)/)[-1]`

`value.replace(cells['Years'].value, "")`

FROM!!!

value.replace(/(?i)\s?to\s?AFTER\s?(?=\d)/, "-LATER")
value.replace(/(?i)\s?prior(\s|(\-)|(\s?to\s?))/, "BEFORE-")
value.replace(/(?i)((?<=\d\d)|((?<=\d\d)(\d\d)?))\sto\,NB/, "-NB")


value.replace(/((?<=\,)|(?<=\-)|(?<=^))(?=[0]\d(\,|\-|$))/, "20")

(\,|\-|^)((0\d)|(1\d)|(20))(\,|\-|$)(2[1-9]|([3-9]\d))(\,|\-|$)

(\,|\-|^)[0]\d(\,|\-).*(21|([3-9]\D))

value.replace(/(?i)((?<=\,)|(?<=^))[a-z0-9]+\-[a-z0-9]+\-[a-z0-9]+(?=(\,|$))/, "20")

# Ann

`if(length(cells['Valuation Standard'].value.find(/(?i)CARVM/))==1, if(length(cells['Method'].value) > 0, "various", "CARVM"), cells['Method'].value)`

`value.replace(/(?i)CARVM/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)NLP/))==1, if(length(cells['Method'].value) > 0, "various", "NLP"), cells['Method'].value)`

`value.replace(/(?i)NLP/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))SPDA(?=\s|$)/))==1, if(length(cells['ImmDef'].value) > 0, "both", "Deferred"), cells['ImmDef'].value)`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))SPDA(?=\s|$)/))==1, cells['OtherMod'].value + "+SinglePremium", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))SPDA(?=\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))SP(?=\s|$)/))==1, cells['OtherMod'].value + "+SinglePremium", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))SP(?=\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))SPIA(?=\s|$)/))==1, if(length(cells['ImmDef'].value) > 0, "both", "Immediate"), cells['ImmDef'].value)`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))SPIA(?=\s|$)/))==1, cells['OtherMod'].value + "+SinglePremium", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))SPIA(?=\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))FPDA(?=\s|$)/))==1, if(length(cells['ImmDef'].value) > 0, "both", "Deferred"), cells['ImmDef'].value)`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))FPDA(?=\s|$)/))==1, cells['OtherMod'].value + "+FlexPremium", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))FPDA(?=\s|$)/, "")`
r`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))FLEX(ible)?\s?PREM(ium)?(?=\s|$)/))==1, cells['OtherMod'].value + "+FlexPremium", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))FLEX(ible)?\s?PREM(ium)?(?=\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))sing(le)?\s?PREM(ium)?(?=\s|$)/))==1, cells['OtherMod'].value + "+SinglePremium", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))sing(le)?\s?PREM(ium)?(?=\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))NFI(?=\s|$)/))==1, cells['OtherMod'].value + "+NFI", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))NFI(?=\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))FI(?=\s|$)/))==1, cells['OtherMod'].value + "+FI", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))FI(?=\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))CIF(?=\s|$)/))==1, cells['OtherMod'].value + "+CIF", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))CIF(?=\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))CRVM(?=\s|$)/))==1, if(length(cells['Method'].value) > 0, "various", "CARVM"), cells['Method'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))CRVM(?=\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))accum(ulation)?(\s|$)/))==1, cells['OtherMod'].value + "+Accu", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))accum(ulation)?(\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))SB(\s|$)/))==1, cells['OtherMod'].value + "+Accu", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))SB(\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))PROJ\.?(\s[a-z])?(\s|$)/))==1, cells['OtherMod'].value + "+Proj", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))PROJ\.?(\s[a-z][a-z]\s?\s?(\d\d\d?\d?)?)?(\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))flexible(?=\s|$)/))==1, if(length(cells['Method'].value) > 0, "various", "Flexible"), cells['Method'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))CRVM(?=\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))RESERVES(?=\s|$)/))==1, if(length(cells['Method'].value) > 0, "various", "Flexible"), cells['Method'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))CRVM(?=\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))certain(?=\s|$)/))==1, if(length(cells['Method'].value) > 0, "various", "Certain"), cells['Method'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))certain(?=\s|$)/, "")`

`value + "+SinglePremium"`

`if(length(cells['Valuation Standard'].value.find(/(?i)(net )?level($|\s|P|\spremium)/))==1, if(length(cells['Method'].value) > 0, "various", "NLP"), cells['Method'].value)`

`value.replace(/(?i)(net )?level($|\s|P|\spremium)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))VM22(?=\s|$)/))==1, if(length(cells['Method'].value) > 0, "various", "VM22"), cells['Method'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))VM22(?=\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))NON(\s|\-)JUMBO(\s|$)/))==1, cells['OtherMod'].value + "+NonJumbo", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))NON(\s|\-)JUMBO(\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))IS(?=\s|$)/))==1, cells['OtherMod'].value + "+IS", cells['OtherMod'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))IS(?=\s|$|\d)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))I(?=\s|$)/))==1, if(length(cells['ImmDef'].value) > 0, "both", "Immediate"), cells['ImmDef'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))I(?=\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))D(?=\s|$)/))==1, if(length(cells['ImmDef'].value) > 0, "both", "Deferred"), cells['ImmDef'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))D(?=\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))B(?=\s|$)/))==1, if(length(cells['ImmDef'].value) > 0, "both", "both"), cells['ImmDef'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))B(?=\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))D(\/|\&)I(?=\s|$)/))==1, if(length(cells['ImmDef'].value) > 0, "both", "both"), cells['ImmDef'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))D(\/|\&)I(?=\s|$)/, "")`

`if(length(cells['Valuation Standard'].value.find(/(?i)((?<=\s)|(?<=^))anxb(?=\s|$)/))==1, if(length(cells['Method'].value) > 0, "various", "ALB"), cells['Bday'].value)`

`value.replace(/(?i)((?<=\s)|(?<=^))anxb(?=\s|$)/, "")`


value.replace(/(?i)((?<=\s)|(?<=^))universal(?=\s|$|\,|\/)/, "")

((?<=\s)|(?<=^))universal life(?=\s|$|\,)

^\s?both\s?$

value + "+VUL"


`sha1(cells['NAIC_no'].value+cells['Mort'].value+cells['Sex'].value+cells['AddLife'].value+cells['Bday'].value+cells['Rate'].value+cells['Method'].value+cells['ImmDef'].value+cells['OtherMod'].value+cells['Years'].value)`

# Ingestion

After cleaning in OpenRefine:

In [10]:
df = CSV.read("C24341-cleaned_6-12-2021.csv", DataFrame)

describe(df)

,variable,mean,min,median,max
,Symbol,Union…,Any,Union…,Any
1,AMB Company Number,7022.11,1850,6812.0,9949
2,rnd,500.106,0,501.0,1000
3,AMB Company Name,,4 Ever Life Insurance Company,,Zurich American Life Insurance Company
4,NAIC Company Number,69824.0,15638,67091.0,99937
5,Year,2010.62,2001,2011.0,2020
6,Line,2.16186e5,100001,200005.0,700041
7,Valuation Standard,,\x16,,~
8,YearsImputed,,2005-2006,,2005-2006
9,OtherMod,,+100%,,+VUL
